In [39]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import glob
import os
import metpy
import numpy as np
from datetime import datetime
import xarray as xr
import metpy.calc as mpcalc
from metpy.units import units
from metpy.calc import dewpoint_from_relative_humidity
from metpy.calc import specific_humidity_from_dewpoint
from pint import UnitRegistry
ureg = UnitRegistry()
import seaborn as sns

In [40]:
# Function to extract epoch from the first few rows of the CSV file
def extract_epoch_from_header(file):
    # Read the first few lines to find the epoch (assuming it's in a comment or a header)
    with open(file, 'r') as f:
        skiprow = f.readline()
        skiprow = skiprow.strip().split()[0]
        for line in f:
            if "startdate:" in line.lower():  # Assuming the word 'epoch' is present in the line
                # Extract the epoch date (this assumes the date is the second item in the line)
                epoch = line.strip().split()[1]
                return int(skiprow)-1, epoch[0:4]+"-"+epoch[4:6]+"-"+epoch[6:8]+" "+epoch[8:10]+":"+epoch[10:12]+":"+epoch[12:14]
    return None  # Return None if no epoch is found

# Function to load and adjust time for each CSV
def load_and_adjust_time(file):
    # Try to extract the epoch from the header or a specific column
    nskiprows,epoch = extract_epoch_from_header(file)
   
    if epoch is None:
        raise ValueError(f"Could not find epoch for file: {file}")
   
    # Load the actual data (skipping header if necessary)
    df = pd.read_csv(file, skiprows=nskiprows,sep="\\s+")  # Adjust skiprows based on where the data starts
   
    # Convert 'time' column to a datetime, assuming it's in days
    # Adjust time column to start from the epoch time found in the file
    df['DATETIME'] = pd.to_timedelta(df['start_time'], unit='D') + pd.Timestamp(epoch)
   
    # Return the adjusted DataFrame
    return df


In [41]:

#set file path


meteo_15_19 = "C:\\YEAR1\\NMVOC\\Data\\data_Meteo\\Meteo_ebas_11_15\\IT0009R.20150101000000.20190912000000.aws.met.1y.1h.lev2.nas"

df = load_and_adjust_time(meteo_15_19)

df.index = df["DATETIME"]

print(df.head())  # Shows the first 5 rows


                            start_time  end_time  wind_speed  \
DATETIME                                                       
2015-01-01 00:00:00.000000    0.000000  0.041667        99.9   
2015-01-01 01:00:00.028800    0.041667  0.083333        99.9   
2015-01-01 01:59:59.971200    0.083333  0.125000        99.9   
2015-01-01 03:00:00.000000    0.125000  0.166667        99.9   
2015-01-01 04:00:00.028800    0.166667  0.208333        99.9   

                            numflag_wind_speed  wind_direction  \
DATETIME                                                         
2015-01-01 00:00:00.000000               0.999           999.9   
2015-01-01 01:00:00.028800               0.999           999.9   
2015-01-01 01:59:59.971200               0.999           999.9   
2015-01-01 03:00:00.000000               0.999           999.9   
2015-01-01 04:00:00.028800               0.999           999.9   

                            numflag_wind_direction  temperature  \
DATETIME             

In [32]:
df.tail(10)  # Shows the last 10 rows

,start_time,end_time,wind_speed,numflag_wind_speed,wind_direction,numflag_wind_direction,temperature,numflag_temperature,relative_humidity,numflag_relative_humidity,pressure,numflag_pressure,global_radiation,numflag_global_radiation,DATETIME
DATETIME,,,,,,,,,,,,,,,
2015-12-31 12:00:00.000000,364.500000,364.541667,4.5,0.0,273.4,0.0,1.3,0.0,24.5,0.0,793.0,0.0,168.1,0.0,2015-12-31 12:00:00.000000
2015-12-31 13:00:00.028800,364.541667,364.583333,6.2,0.0,281.9,0.0,0.5,0.0,50.8,0.0,792.9,0.0,114.9,0.0,2015-12-31 13:00:00.028800
2015-12-31 13:59:59.971200,364.583333,364.625000,4.2,0.0,241.8,0.0,-0.5,0.0,53.4,0.0,793.3,0.0,45.0,0.0,2015-12-31 13:59:59.971200
2015-12-31 15:00:00.000000,364.625000,364.666667,2.4,0.0,221.4,0.0,-0.9,0.0,73.9,0.0,793.6,0.0,6.6,0.0,2015-12-31 15:00:00.000000
2015-12-31 16:00:00.028800,364.666667,364.708333,1.9,0.0,14.7,0.0,0.4,0.0,57.4,0.0,793.5,0.0,0.0,0.0,2015-12-31 16:00:00.028800
2015-12-31 16:59:59.971200,364.708333,364.750000,2.1,0.0,12.5,0.0,0.9,0.0,53.4,0.0,793.4,0.0,0.0,0.0,2015-12-31 16:59:59.971200
2015-12-31 18:00:00.000000,364.750000,364.791667,3.3,0.0,34.4,0.0,0.6,0.0,43.4,0.0,793.4,0.0,0.0,0.0,2015-12-31 18:00:00.000000
2015-12-31 19:00:00.028800,364.791667,364.833333,6.0,0.0,25.7,0.0,0.3,0.0,28.0,0.0,793.4,0.0,0.0,0.0,2015-12-31 19:00:00.028800
2015-12-31 19:59:59.971200,364.833333,364.875000,4.7,0.0,29.8,0.0,0.4,0.0,27.0,0.0,793.3,0.0,0.0,0.0,2015-12-31 19:59:59.971200


In [42]:
  
#remove lines depending on flags
df = df[df.numflag_wind_speed != 0.999]
df = df[df.numflag_relative_humidity != 0.999]
df = df[df.numflag_temperature != 0.999]
df = df[df.numflag_pressure != 0.999]

df = df[df.numflag_wind_speed != -0.999]
df = df[df.numflag_relative_humidity != -0.999]
df = df[df.numflag_temperature != -0.999]
df = df[df.numflag_pressure != -0.999]

df = df[df["relative_humidity"] != -999.9]
df = df[df["pressure"] != -999.9]
df = df[df["temperature"] != -999.9]
df = df[df["wind_speed"] != -999.9]

df = df[df.numflag_wind_speed != 0.899]
df = df[df.numflag_relative_humidity != 0.899]
df = df[df.numflag_temperature != 0.899]

df = df[df.numflag_wind_speed != 0.456]
df = df[df.numflag_relative_humidity != 0.456]
df = df[df.numflag_temperature != 0.456]

df = df[df.numflag_wind_speed != 0.260]
df = df[df.numflag_relative_humidity != 0.260]
df = df[df.numflag_temperature != 0.260]

df = df[df.numflag_wind_speed != 0.259]
df = df[df.numflag_relative_humidity != 0.259]
df = df[df.numflag_temperature != 0.259]

df = df[df.numflag_wind_speed != 0.256]
df = df[df.numflag_relative_humidity != 0.255]
df = df[df.numflag_temperature != 0.255]

df = df[df.numflag_wind_speed != 99999.99]


df.index = df["DATETIME"]
#remove useless columns
df = df.drop(columns=["numflag_wind_speed"])
df = df.drop(columns=["numflag_relative_humidity"])
df = df.drop(columns=["numflag_temperature"])
df = df.drop(columns=["start_time"])
df = df.drop(columns=["end_time"])
df=df.drop(columns=["global_radiation"])
df=df.drop(columns=["numflag_global_radiation"])
df=df.drop(columns=["numflag_wind_direction"])
df=df.drop(columns=["numflag_pressure"])
df=df.drop(columns=["wind_direction"])

print(df.head())  # Shows the first 5 rows


                            wind_speed  temperature  relative_humidity  \
DATETIME                                                                 
2015-01-02 13:00:00.028800         9.6          1.9               69.5   
2015-01-02 13:59:59.971200        12.8          2.3               59.0   
2015-01-02 15:00:00.000000        13.7          2.7               53.9   
2015-01-02 16:00:00.028800         9.7          2.5               55.2   
2015-01-02 16:59:59.971200        10.2          2.8               51.2   

                            pressure                   DATETIME  
DATETIME                                                         
2015-01-02 13:00:00.028800     800.5 2015-01-02 13:00:00.028800  
2015-01-02 13:59:59.971200     800.3 2015-01-02 13:59:59.971200  
2015-01-02 15:00:00.000000     800.2 2015-01-02 15:00:00.000000  
2015-01-02 16:00:00.028800     800.2 2015-01-02 16:00:00.028800  
2015-01-02 16:59:59.971200     800.3 2015-01-02 16:59:59.971200  


In [ ]:
for col in df.columns:
    print(col)

In [43]:
#loop for rounding to the hours
df["DATETIME"] = [i.round("h") for i in df["DATETIME"]]
df

,wind_speed,temperature,relative_humidity,pressure,DATETIME
DATETIME,,,,,
2015-01-02 13:00:00.028800,9.6,1.9,69.5,800.5,2015-01-02 13:00:00
2015-01-02 13:59:59.971200,12.8,2.3,59.0,800.3,2015-01-02 14:00:00
2015-01-02 15:00:00.000000,13.7,2.7,53.9,800.2,2015-01-02 15:00:00
2015-01-02 16:00:00.028800,9.7,2.5,55.2,800.2,2015-01-02 16:00:00
2015-01-02 16:59:59.971200,10.2,2.8,51.2,800.3,2015-01-02 17:00:00
...,...,...,...,...,...
2015-12-31 16:59:59.971200,2.1,0.9,53.4,793.4,2015-12-31 17:00:00
2015-12-31 18:00:00.000000,3.3,0.6,43.4,793.4,2015-12-31 18:00:00
2015-12-31 19:00:00.028800,6.0,0.3,28.0,793.4,2015-12-31 19:00:00


In [44]:

df.index = df["DATETIME"]
df=df.drop(columns=["DATETIME"])
df

,wind_speed,temperature,relative_humidity,pressure
DATETIME,,,,
2015-01-02 13:00:00,9.6,1.9,69.5,800.5
2015-01-02 14:00:00,12.8,2.3,59.0,800.3
2015-01-02 15:00:00,13.7,2.7,53.9,800.2
2015-01-02 16:00:00,9.7,2.5,55.2,800.2
2015-01-02 17:00:00,10.2,2.8,51.2,800.3
...,...,...,...,...
2015-12-31 17:00:00,2.1,0.9,53.4,793.4
2015-12-31 18:00:00,3.3,0.6,43.4,793.4
2015-12-31 19:00:00,6.0,0.3,28.0,793.4


In [ ]:
df.tail(10)  # Shows the last 10 rows

In [45]:
save_dir="C:\\YEAR1\\NMVOC\\Data\\data_Meteo\\"    

# Save final  DataFrame
output_file = os.path.join(save_dir, "processed_met_15_19.csv")
df.to_csv(output_file, index=True)

print(f"Saved final data to: {output_file}")

Saved final data to: C:\YEAR1\NMVOC\Data\data_Meteo\processed_met_15_19.csv


In [46]:


# Define file paths
file1 ="C:\\YEAR1\\NMVOC\\Data\\data_Meteo\\processed_met_11_14.csv"

file2 ="C:\\YEAR1\\NMVOC\\Data\\data_Meteo\\processed_met_15_19.csv"


# Read CSV files
df11_14 = pd.read_csv(file1)
df15_19 = pd.read_csv(file2)

# Concatenate files
final_df = pd.concat([df11_14, df15_19], ignore_index=True)  # Resets index after concatenation

# Save the combined data
output_file = "C:\\YEAR1\\NMVOC\\Data\\data_Meteo\\processed_met_11_15.csv"
final_df.to_csv(output_file, index=False)


print("Concatenation complete. Saved as 'concatenated_output.csv'.")


Concatenation complete. Saved as 'concatenated_output.csv'.


In [47]:
print(final_df.head())  # Shows the first 5 rows

              DATETIME  wind_speed  temperature  relative_humidity  pressure
0  2011-01-18 12:00:00        6.50         4.40              33.65    793.95
1  2011-01-18 13:00:00        7.45         4.45              33.60    793.45
2  2011-01-18 14:00:00        7.25         4.55              32.10    793.10
3  2011-01-18 15:00:00        7.65         4.25              33.40    793.00
4  2011-01-18 16:00:00        8.70         3.50              32.45    793.05


In [48]:
final_df.tail(10)  # Shows the last 10 rows

,DATETIME,wind_speed,temperature,relative_humidity,pressure
42305,2015-12-31 12:00:00,4.5,1.3,24.5,793.0
42306,2015-12-31 13:00:00,6.2,0.5,50.8,792.9
42307,2015-12-31 14:00:00,4.2,-0.5,53.4,793.3
42308,2015-12-31 15:00:00,2.4,-0.9,73.9,793.6
42309,2015-12-31 16:00:00,1.9,0.4,57.4,793.5
42310,2015-12-31 17:00:00,2.1,0.9,53.4,793.4
42311,2015-12-31 18:00:00,3.3,0.6,43.4,793.4
42312,2015-12-31 19:00:00,6.0,0.3,28.0,793.4
42313,2015-12-31 20:00:00,4.7,0.4,27.0,793.3
42314,2015-12-31 21:00:00,4.2,-0.1,22.1,793.2
